In [1]:
import argparse
import httplib2
import time
from apiclient.discovery import build
from oauth2client import client
from oauth2client import file
from oauth2client import tools


In [2]:
def get_service(api_name, api_version, scope, client_secrets_path):
  """Get a service that communicates to a Google API.

  Args:
    api_name: string The name of the api to connect to.
    api_version: string The api version to connect to.
    scope: A list of strings representing the auth scopes to authorize for the
      connection.
    client_secrets_path: string A path to a valid client secrets file.

  Returns:
    A service that is connected to the specified API.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      client_secrets_path, scope=scope,
      message=tools.message_if_missing(client_secrets_path))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage(api_name + '.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  service = build(api_name, api_version, http=http)

  return service


In [3]:
def main():
  # Define the auth scopes to request.
  scope = ['https://www.googleapis.com/auth/analytics.readonly']

  # Authenticate and construct service.
  service = get_service('analytics', 'v3', scope, 'client_secret.json')
  
  return service

In [4]:
service = main()

In [5]:
# 1. Create and Execute a Real Time Report
# An application can request real-time data by calling the get method on the Analytics service object.
# The method requires an ids parameter which specifies from which view (profile) to retrieve data.
# For example, the following code requests real-time data for view (profile) ID 56789.

try:
  service.data().realtime().get(
      ids='ga:91998326',
      metrics='rt:activeUsers',
      dimensions='rt:medium').execute()

except TypeError, error:
  # Handle errors in constructing a query.
  print ('There was an error in constructing your query : %s' % error)

# except HttpError, error:
#   # Handle API errors.
#   print ('Arg, there was an API error : %s : %s' %
#          (error.resp.status, error._get_reason()))


# 2. Print out the Real-Time Data
# The components of the report can be printed out as follows:

def print_realtime_report(results):
  print '**Real-Time Report Response**' 
  print_report_info(results)
  print_query_info(results.get('query'))
  print_profile_info(results.get('profileInfo'))
  print_column_headers(results.get('columnHeaders'))
  print_data_table(results)
  print_totals_for_all_results(results)

def print_data_table(results):
  print 'Data Table:'
  # Print headers.
  output = []
  for header in results.get('columnHeaders'):
    output.append('%30s' % header.get('name'))
  print ''.join(output)
  # Print rows.
  if results.get('rows', []):
    for row in results.get('rows'):
      output = []
      for cell in row:
        output.append('%30s' % cell)
      print ''.join(output)
  else:
    print 'No Results Found'

def print_column_headers(headers):
  print 'Column Headers:'
  for header in headers:
    print 'Column name           = %s' % header.get('name')
    print 'Column Type           = %s' % header.get('columnType')
    print 'Column Data Type      = %s' % header.get('dataType')

def print_query_info(query):
  if query:
    print 'Query Info:'
    print 'Ids                   = %s' % query.get('ids')
    print 'Metrics:              = %s' % query.get('metrics')
    print 'Dimensions            = %s' % query.get('dimensions')
    print 'Sort                  = %s' % query.get('sort')
    print 'Filters               = %s' % query.get('filters')
    print 'Max results           = %s' % query.get('max-results')

def print_profile_info(profile_info):
  if profile_info:
    print 'Profile Info:'
    print 'Account ID            = %s' % profile_info.get('accountId')
    print 'Web Property ID       = %s' % profile_info.get('webPropertyId')
    print 'Profile ID            = %s' % profile_info.get('profileId')
    print 'Profile Name          = %s' % profile_info.get('profileName')
    print 'Table Id              = %s' % profile_info.get('tableId')

def print_report_info(results):
  print 'Kind                    = %s' % results.get('kind')
  print 'ID                      = %s' % results.get('id')
  print 'Self Link               = %s' % results.get('selfLink')
  print 'Total Results           = %s' % results.get('totalResults')

def print_totals_for_all_results(results):
  totals = results.get('totalsForAllResults')
  for metric_name, metric_total in totals.iteritems():
    print 'Metric Name  = %s' % metric_name
    print 'Metric Total = %s' % metric_total

In [6]:
def looper():
    results = service.data().realtime().get(
          ids='ga:91998326',
          metrics='rt:activeUsers',
          dimensions='rt:medium').execute()

    print_realtime_report(results)

In [7]:
while True:
    looper()
    time.sleep(15)

**Real-Time Report Response**
Kind                    = analytics#realtimeData
ID                      = https://www.googleapis.com/analytics/v3/data/realtime?ids=ga:91998326&dimensions=rt:medium&metrics=rt:activeUsers
Self Link               = https://www.googleapis.com/analytics/v3/data/realtime?ids=ga:91998326&dimensions=rt:medium&metrics=rt:activeUsers
Total Results           = 12
Query Info:
Ids                   = ga:91998326
Metrics:              = [u'rt:activeUsers']
Dimensions            = rt:medium
Sort                  = None
Filters               = None
Max results           = 1000
Profile Info:
Account ID            = 55400931
Web Property ID       = UA-55400931-2
Profile ID            = 91998326
Profile Name          = *** Very - All Data ***
Table Id              = realtime:91998326
Column Headers:
Column name           = rt:medium
Column Type           = DIMENSION
Column Data Type      = STRING
Column name           = rt:activeUsers
Column Type           = METRIC
Column

KeyboardInterrupt: 